In [37]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.trend import AroonIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
#from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator
import time

from IPython.display import clear_output
from datetime import datetime
from datetime import date
from datetime import timedelta
today = date.today()

start=(datetime.now())
sip500=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
sip400=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0].iloc[:,1])
sip600=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1].iloc[:,1])
payload=np.append(sip500,sip400)
payload=np.append(payload,sip600)
payload=np.unique(payload)
payload=payload[payload!='BRK.B']
payload=payload[payload!='JW.A']
payload=payload[payload!='BF.B']
monday = today - timedelta(days = today.weekday())
#print(payload)
#print(monday)

class algo:
    start="2010-01-04"
    end=today
    interval='1wk'

    dff=pd.DataFrame()
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        try:
            df=(yf.download(self.stock,algo.start,algo.end,interval=algo.interval).dropna())

            #df['log_current']=np.log(df.Close/df.Open)
            df['stock']=self.stock
            #output = stock data from start to end
            clear_output(wait=False)
        except:
            pass
    
        return(df)
    
    
    def stockloop():
        l1=0
        l2=500
        #l2=len(payload)
        dff=algo.dff
        boom=pd.DataFrame()
        for i in range (l1,l2):
            try:
                rawdata=algo(payload[i]).getdata()
                dff=pd.concat([dff,rawdata])
                print(i)
            except Exception as e:
                print(e)
                pass
        dff.set_index([dff.stock,dff.index],inplace=True)
        return(dff)

dff=algo.stockloop()
#dff.to_csv('raw.csv')
uniq=(dff.index.unique(level=0))

finish = datetime.now()
print('Duration: {}'.format(finish - start))
time.sleep(6000)

499
Duration: 0:02:10.766535


KeyboardInterrupt: 

In [39]:
start=(datetime.now())
#print(dff)
class algo2:
    dfmass=pd.DataFrame()
    thr1=1
    thr2=99
    end=['2022-01-01','2021-01-01','2020-01-01','2019-01-01','2018-01-01','2017-01-01','2016-01-01','2015-01-01','2014-01-01']
    def __init__(self,stock):
        self.stock=stock

    def indicators(self):
        df=self.stock
        df['DR']=(((df.Close/df.Open)-1)*df.Volume)
        df['rsi']=RSIIndicator(close=df.Close,window=12).rsi()
        df['FI']=ForceIndexIndicator(close=df.Close,volume=df.Volume,window=12).force_index()
        df['MACD']=MACD(close=df.Close).macd()
        df['MACDh']=MACD(close=df.Close).macd_diff()
        df['condition1']=df.DR
        df['condition2']=df.FI
        df['condition3']=df.MACD
        df['condition4']=df.MACDh
        df['condition5']=df.rsi
        df=df.dropna()

        #output = stock data with indicators
        return(df)

        #return(dict21,dict22)
    def trade(self):
        df=self.stock
        dff=pd.DataFrame()

        df['trade_pass']=np.select(

        [(df.condition1<np.nanpercentile(df.condition1,5))
         &(df.condition2<np.nanpercentile(df.condition2,5))
         &(df.condition3<np.nanpercentile(df.condition3,5))
         &(df.condition4<np.nanpercentile(df.condition4,5))
         &(df.condition5<np.nanpercentile(df.condition5,5))
         ,

         (df.condition1>1000000)
        ]

        ,[1,-1],default=0)

        dff=pd.concat([dff,df])
        return(dff)
    def stockloop():
        boom=pd.DataFrame()
        for i in uniq:
            try:
                rawdata=(dff.loc[i])
                ta=algo2(rawdata).indicators()
                trades=algo2(ta).trade()
                boom=pd.concat([boom,trades])
                clear_output(wait=False)
                print(i)
            except: 
                pass
        return(boom)


boom=algo2.stockloop()
#print(boom)
boom.to_csv('all_data.csv')

finish = datetime.now()
print('Duration: {}'.format(finish - start))

FDP
Duration: 0:00:38.272091


In [40]:
boomnow=(boom[boom.index==pd.Timestamp(monday)])
print(boomnow[boomnow.trade_pass==1])

Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume, stock, DR, rsi, FI, MACD, MACDh, condition1, condition2, condition3, condition4, condition5, trade_pass]
Index: []
